In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import sys
import os
import openpyxl
import xlrd

In [2]:
# !pip install xlrd
# !pip install openpyxl

# Read in the Data

In [3]:
import pandas as pd

def preprocess_and_filter_dates(df, date_column, drop_columns=None, **rename_dict):
    """
    Preprocesses the given date column in the DataFrame.
    Converts it to 'Month Year' format, checks for the full date range from Jan 2013 to Oct 2024,
    sorts the DataFrame, filters the data to keep only records within the date range,
    renames the date column to 'Date', renames other columns using provided keyword arguments,
    drops specified columns, and resets the index of the DataFrame.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame
    - date_column (str): The name of the date column to be processed and renamed to 'Date'
    - drop_columns (list, optional): List of columns to drop from the DataFrame
    - **rename_dict: Keyword arguments for renaming columns, e.g., old_name='new_name'
    
    Returns:
    - pd.DataFrame: The sorted, filtered, renamed, and indexed-reset DataFrame
    """
    # Convert the date column to datetime and extract Month and Year
    df[date_column] = pd.to_datetime(df[date_column]).dt.to_period('M').dt.to_timestamp()
    
    # Rename the date column to 'Date'
    df = df.rename(columns={date_column: 'Date'})
    
    # Define the expected date range
    start_date = pd.Timestamp('2013-01-01')
    end_date = pd.Timestamp('2024-08-01')
    
    # Check if the full date range is present
    all_dates = pd.date_range(start=start_date, end=end_date, freq='MS')
    if not all_dates.isin(df['Date'].unique()).all():
        raise ValueError("Warning: The complete date range from Jan 2013 to Oct 2024 is not present in the data.")
    
    # Sort the DataFrame by the 'Date' column in ascending order
    df = df.sort_values(by='Date')
    
    # Filter the data to keep only records within the date range
    df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    # Rename other columns using the keyword arguments
    if rename_dict:
        df = df.rename(columns=rename_dict)
    
    # Drop specified columns if drop_columns is provided
    if drop_columns:
        df = df.drop(columns=drop_columns)
    
    # Reset the index of the DataFrame
    df = df.reset_index(drop=True)
    
    return df


## Averge Hourly Earning

In [4]:
df_avg_hourly_earning = pd.read_csv("Average Hourly Earnings of All Employees, Total Private_FRED.csv")
df_avg_hourly_earning.head(10)

,DATE,CES0500000003
0,2006-03-01,20.05
1,2006-04-01,20.15
2,2006-05-01,20.13
3,2006-06-01,20.23
4,2006-07-01,20.29
5,2006-08-01,20.32
6,2006-09-01,20.41
7,2006-10-01,20.43
8,2006-11-01,20.49
9,2006-12-01,20.57


In [5]:
df_Avg_Hourly_Earning = preprocess_and_filter_dates(df_avg_hourly_earning,
                                                    'DATE',
                                                   CES0500000003='Average_Hourly_Earning')
df_Avg_Hourly_Earning.head(3)

,Date,Average_Hourly_Earning
0,2013-01-01,23.75
1,2013-02-01,23.78
2,2013-03-01,23.80


In [6]:
df_Avg_Hourly_Earning.tail(3)

,Date,Average_Hourly_Earning
137,2024-06-01,34.99
138,2024-07-01,35.07
139,2024-08-01,35.22


## Baltic Dry Index Historical Data 

In [7]:
# Read the CSV file and convert specific columns to numeric
df2 = pd.read_csv(
    'Baltic Dry Index Historical Data monthly(2012-present)_Investing.com.csv',
    converters={
        'Price': pd.to_numeric,
        'Open': pd.to_numeric,
        'High': pd.to_numeric,
        'Low': pd.to_numeric
    }
)

df2.head(4)

,Date,Price,Open,High,Low,Vol.,Change %
0,11/8/2024,1495.0,1495.0,1495.0,1495.0,NaN,0.030
1,11/7/2024,1451.0,1451.0,1451.0,1451.0,NaN,0.017
2,11/6/2024,1427.0,1427.0,1427.0,1427.0,NaN,0.016
3,11/5/2024,1405.0,1405.0,1405.0,1405.0,NaN,0.023


In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      2998 non-null   object 
 1   Price     2998 non-null   float64
 2   Open      2998 non-null   float64
 3   High      2998 non-null   float64
 4   Low       2998 non-null   float64
 5   Vol.      0 non-null      float64
 6   Change %  2998 non-null   float64
dtypes: float64(6), object(1)
memory usage: 164.1+ KB


In [9]:
# # Convert numeric columns to float
# df2[['Price', 'Open', 'High', 'Low']] = df2[['Price', 'Open', 'High', 'Low']].apply(pd.to_numeric, errors='coerce')

# # Remove the '%' sign from 'Change %' and convert to float
# df2['Change %'] = df2['Change %'].str.replace('%', '').astype(float)

### Converting from a Daily Level dataset to a Montly level dataset

In [10]:
df2['Date'] = pd.to_datetime(df2['Date'])  # Convert Date to datetime

# Extract the year and month from the Date column
df2['YearMonth'] = df2['Date'].dt.to_period('M')

# Group by YearMonth and aggregate
monthly_df_baltic = df2.groupby('YearMonth').agg({
    'Price': 'mean',            # Average Price for the month
    'Open': 'first',            # Opening Price of the first day of the month
    'High': 'max',              # Highest Price in the month
    'Low': 'min',               # Lowest Price in the month
    'Vol.': 'sum',              # Total volume for the month (if volume data is available)
    'Change %': 'mean'          # Average Change % for the month
}).reset_index()                # Reset index to keep YearMonth as a column

# Convert YearMonth back to a datetime object in the format you prefer
monthly_df_baltic['Date'] = monthly_df_baltic['YearMonth'].dt.to_timestamp()

# Drop the YearMonth column, if you only want to keep the Date
monthly_df_baltic = monthly_df_baltic.drop(columns=['YearMonth'])

# Display the first few rows of the monthly data
print(monthly_df_baltic.head())

         Price    Open    High     Low  Vol.  Change %       Date
0  1026.000000  1026.0  1026.0  1026.0   0.0 -0.016000 2012-10-01
1  1024.636364  1086.0  1104.0   916.0   0.0  0.002773 2012-11-01
2   855.687500   699.0  1077.0   699.0   0.0 -0.026937 2012-12-01
3   770.590909   760.0   838.0   698.0   0.0  0.004045 2013-01-01
4   744.950000   757.0   757.0   735.0   0.0 -0.000250 2013-02-01


In [11]:
monthly_df_baltic.head(20)

,Price,Open,High,Low,Vol.,Change %,Date
0,1026.000000,1026.0,1026.0,1026.0,0.0,-0.016000,2012-10-01
1,1024.636364,1086.0,1104.0,916.0,0.0,0.002773,2012-11-01
2,855.687500,699.0,1077.0,699.0,0.0,-0.026937,2012-12-01
3,770.590909,760.0,838.0,698.0,0.0,0.004045,2013-01-01
4,744.950000,757.0,757.0,735.0,0.0,-0.000250,2013-02-01
5,876.100000,910.0,935.0,776.0,0.0,0.009400,2013-03-01
6,874.000000,863.0,896.0,856.0,0.0,-0.002476,2013-04-01
7,851.380952,809.0,892.0,809.0,0.0,-0.003095,2013-05-01
8,940.500000,1171.0,1171.0,801.0,0.0,0.018700,2013-06-01
9,1123.130435,1062.0,1179.0,1062.0,0.0,-0.004174,2013-07-01


In [12]:
monthly_df_baltic.columns

Index(['Price', 'Open', 'High', 'Low', 'Vol.', 'Change %', 'Date'], dtype='object')

In [13]:
monthly_df_baltic.rename(columns={'Change %': 'Change_Percent'}, inplace=True)

In [14]:
df_Global_Shipping_Cost = preprocess_and_filter_dates(monthly_df_baltic,
                                                      date_column='Date',
                                                      Change_Percent='Trade_Volume_Pct_Change_Index_Value',
                                                      drop_columns=['Price', 'Open', 'High', 'Low', 'Vol.'] )
df_Global_Shipping_Cost.head(3)

,Trade_Volume_Pct_Change_Index_Value,Date
0,0.004045,2013-01-01
1,-0.000250,2013-02-01
2,0.009400,2013-03-01


In [15]:
df_Global_Shipping_Cost.tail(3)

,Trade_Volume_Pct_Change_Index_Value,Date
137,0.006300,2024-06-01
138,-0.007652,2024-07-01
139,0.003048,2024-08-01


## Monthly Crude Oil Prices

In [16]:
df3 = pd.read_excel('Crude Oil Prices Monthly_US energy information administration(EIA).xls', sheet_name='Data 1')
df3.head()

,Date,U.S. Crude Oil First Purchase Price (Dollars per Barrel)
0,1974-01-15,6.95
1,1974-02-15,6.87
2,1974-03-15,6.77
3,1974-04-15,6.77
4,1974-05-15,6.87


In [17]:
df3.rename(columns={'U.S. Crude Oil First Purchase Price (Dollars per Barrel)':'US_Crude_Oil_Prices_Dollar_Per_Barrel'}, inplace=True)

In [18]:
df_Crude_Oil_Prices_Monthly = preprocess_and_filter_dates(df3,
                                                      date_column='Date')

In [19]:
print(df_Crude_Oil_Prices_Monthly.head(3))
print('\n')
print(df_Crude_Oil_Prices_Monthly.tail(3))

        Date  US_Crude_Oil_Prices_Dollar_Per_Barrel
0 2013-01-01                                  95.00
1 2013-02-01                                  95.01
2 2013-03-01                                  95.54


          Date  US_Crude_Oil_Prices_Dollar_Per_Barrel
137 2024-06-01                                  77.45
138 2024-07-01                                  79.07
139 2024-08-01                                  74.97


## Federal Funds Effective Rate Monthly

In [20]:
df4 = pd.read_csv('Federal Funds Effective Rate_FRED.csv')
df4.head(5)

,DATE,FEDFUNDS
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.07
3,1954-10-01,0.85
4,1954-11-01,0.83


In [21]:
df_Federal_Funds_Rate= preprocess_and_filter_dates(df4,
                                                      date_column='DATE',
                                                         FEDFUNDS='Federal_Rates_Monthly')

In [22]:
print(df_Federal_Funds_Rate.head(3))
print('\n')
print(df_Federal_Funds_Rate.tail(3))

        Date  Federal_Rates_Monthly
0 2013-01-01                   0.14
1 2013-02-01                   0.15
2 2013-03-01                   0.14


          Date  Federal_Rates_Monthly
137 2024-06-01                   5.33
138 2024-07-01                   5.33
139 2024-08-01                   5.33


## Inflation Data Monthly

In [23]:
df5 = pd.read_csv('Inflation_data_monthly(2013-present)_Bureau of Labour Statistics.csv')
df5.head(5)

,Series ID,Year,Period,Label,Value,1-Month Net Change,1-Month % Change
0,CUUR0000SA0,2013,M01,2013 Jan,230.280,NaN,0.3
1,CUUR0000SA0,2013,M02,2013 Feb,232.166,NaN,0.8
2,CUUR0000SA0,2013,M03,2013 Mar,232.773,NaN,0.3
3,CUUR0000SA0,2013,M04,2013 Apr,232.531,NaN,-0.1
4,CUUR0000SA0,2013,M05,2013 May,232.945,NaN,0.2


In [24]:
df5.columns

Index(['Series ID', 'Year', 'Period', 'Label', 'Value', '1-Month Net Change',
       '1-Month % Change'],
      dtype='object')

For the econometric VAR model, **the most relevant column would likely be the "Value" column (i.e., the CPI Value), rather than the "1-Month % Change"**.

Here's Why:
Using the CPI Value ("Value" Column):

The CPI Value represents the overall price level and is essential for capturing the long-term trends and dynamics in the economy, which is crucial for a VAR model that examines relationships between multiple economic indicators over time.
VAR models are designed to analyze the relationships among multiple time series variables, so having the actual CPI values allows you to model the interaction between the level of inflation and other economic variables (e.g., crude oil prices, unemployment rate, etc.) over time.
You can always calculate inflation rates or percentage changes from the CPI values if needed for additional analysis.
Why Not Use "1-Month % Change" Directly:

The "1-Month % Change" column only shows short-term changes in the CPI and may miss out on the broader economic trends that are important for modeling long-term relationships in a VAR framework.
Using the actual CPI values provides more context and is generally preferred for modeling purposes.

In [25]:
df_inflation_cpi = preprocess_and_filter_dates(df5,
                                               date_column='Label',
                                               Value='CPI_Value_Inflation',
                                               drop_columns=['Series ID', 'Year', 'Period', '1-Month Net Change', '1-Month % Change'])

C:\Users\anmol\AppData\Local\Temp\ipykernel_7460\1398366478.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_column] = pd.to_datetime(df[date_column]).dt.to_period('M').dt.to_timestamp()


In [26]:
print(df_inflation_cpi.head(3))
print(df_inflation_cpi.tail(3))

        Date  CPI_Value_Inflation
0 2013-01-01              230.280
1 2013-02-01              232.166
2 2013-03-01              232.773
          Date  CPI_Value_Inflation
137 2024-06-01              314.175
138 2024-07-01              314.540
139 2024-08-01              314.796


## Nominal Broad US Dollar Index

**Broad Basket**: The index is "broad" because it includes a wide range of currencies from both developed and emerging economies that are significant trading partners of the United States. This gives a more comprehensive picture of the dollar's value compared to just focusing on a small number of currencies.
**Purpose**:The index is used to understand the overall strength or weakness of the U.S. dollar in the global market.
It is a key indicator for analyzing the impact of exchange rate fluctuations on trade, inflation, and overall economic performance.
**Impact on the Economy**:A stronger U.S. dollar makes imports cheaper but can make U.S. exports more expensive for foreign buyers, potentially reducing demand for American goods abroad.
A weaker U.S. dollar makes exports more competitive but increases the cost of imports, which can contribute to higher inflation domestically.
**Relevance to the Econometric Model**: If you are studying the impact of international factors (like the Russia-Ukraine conflict) on the U.S. economy, the Nominal Broad U.S. Dollar Index can be an important variable. It helps capture how changes in the value of the dollar influence trade balances, import/export prices, and overall economic conditions, which are crucial for understanding the broader economic impacts of global events.

In [27]:
df6 = pd.read_csv('Nominal Broad U.S. Dollar Index_FRED.csv')
df6.head(3)

,DATE,TWEXBGSMTH
0,2006-01-01,100.0000
1,2006-02-01,100.2112
2,2006-03-01,100.4281


In [28]:
df_nominal_broad_usd_index = preprocess_and_filter_dates(df6,
                                               date_column='DATE',
                                               TWEXBGSMTH='Nominal_Broad_US_Dollar_Index')
                                               # drop_columns=['Series ID', 'Year', 'Period', '1-Month Net Change', '1-Month % Change'])

In [29]:
print(df_nominal_broad_usd_index.head(3))
print(df_nominal_broad_usd_index.tail(3))

        Date  Nominal_Broad_US_Dollar_Index
0 2013-01-01                        90.8255
1 2013-02-01                        91.5509
2 2013-03-01                        92.4132
          Date  Nominal_Broad_US_Dollar_Index
137 2024-06-01                       124.0309
138 2024-07-01                       123.7208
139 2024-08-01                       122.8802


## Producer Price Index by Commodity (All Commodities)

The Producer Price Index (PPI) by Commodity: All Commodities is an economic indicator that measures the average change over time in the selling prices received by domestic producers for their output. Unlike the Consumer Price Index (CPI), which measures price changes from the perspective of the consumer, the PPI captures price changes from the perspective of the producer.

Key Points About the PPI by Commodity: All Commodities

**Scope**:The index covers a broad range of commodities produced in the United States, including goods like energy products, food, metals, and other raw materials.It provides an overall measure of inflation at the producer level, reflecting the prices that producers receive for their goods before they reach consumers.

**Frequency**:The index is typically reported on a monthly basis, making it suitable for analyzing short-term and long-term trends in producer prices.

**Uses**:Inflation Analysis: The PPI is used to monitor inflationary pressures in the economy, especially in the early stages of the production process. Rising producer prices often signal future increases in consumer prices.
Economic Forecasting: Economists use the PPI to forecast inflation and assess economic conditions. It provides insights into the cost pressures facing businesses, which can affect pricing strategies and profit margins.
Policy Making: Central banks and policymakers monitor the PPI to inform monetary policy decisions, as changes in producer prices can influence overall inflation and economic stability.

**Relevance to Your Econometric Model**
The PPI by Commodity: All Commodities can be an important variable in your VAR model if you are studying the impact of global events (like the Russia-Ukraine conflict) on inflation and economic conditions in the U.S.
Changes in producer prices can be influenced by global commodity markets, supply chain disruptions, and international trade dynamics, making the PPI a key indicator of economic pressures from external shocks.

In [30]:
df7 = pd.read_csv('Producer Price Index by Commodity All Commodities (monthly)_FRED.csv')
df7.head(4)

,DATE,PPIACO
0,1913-01-01,12.1
1,1913-02-01,12.0
2,1913-03-01,12.0
3,1913-04-01,12.0


In [31]:
df_producer_price_index_all_commodities = preprocess_and_filter_dates(df7,
                                               date_column='DATE',
                                               PPIACO='producer_price_index_all_commodities')

In [32]:
print(df_producer_price_index_all_commodities.head(3))
print(df_producer_price_index_all_commodities.tail(3))

        Date  producer_price_index_all_commodities
0 2013-01-01                                 202.5
1 2013-02-01                                 204.3
2 2013-03-01                                 204.0
          Date  producer_price_index_all_commodities
137 2024-06-01                               255.870
138 2024-07-01                               257.337
139 2024-08-01                               255.487


## S&P 500 Data

**Relevance of the S&P 500**
The S&P 500 Index is a widely followed stock market index that represents the performance of 500 of the largest publicly traded companies in the United States. It is often used as a barometer for the overall health of the U.S. stock market and economy.Changes in the S&P 500 Index can be influenced by various economic factors, such as interest rates, inflation, and geopolitical events.

**Use in Econometric Analysis**
If you're analyzing the impact of economic or geopolitical events (like the Russia-Ukraine conflict) on the U.S. economy, the S&P 500 Index is a relevant variable. It reflects investor sentiment, market volatility, and the financial impact of external shocks. The **"Adj Close"** column is especially useful for analyzing the long-term performance of the index, as it accounts for factors that could distort the raw closing price.

In [33]:
df8 = pd.read_excel('S&P 500_Yahoo.xlsx')
df8.head(4)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2024-11-11,6008.86,6017.31,5986.69,6001.35,6001.35,2714209000
1,2024-11-01,5723.22,6017.31,5696.51,6001.35,6001.35,31421830000
2,2024-10-01,5757.73,5878.46,5674.00,5705.45,5705.45,82412430000
3,2024-09-01,5623.89,5767.37,5402.62,5762.48,5762.48,79564830000


In [34]:
df8.rename(columns={'Adj Close':'Adj_Close'}, inplace=True)

In [35]:
df8.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume'], dtype='object')

In [36]:
df_sp500 = preprocess_and_filter_dates(df8,
                                       date_column='Date',
                                        Adj_Close='Adjusted_Closing_Price',
                                        drop_columns=['Open', 'High', 'Low', 'Close', 'Volume'])

In [37]:
print(df_sp500.head(3))
print(df_sp500.tail(3))

        Date  Adjusted_Closing_Price
0 2013-01-01                 1498.11
1 2013-02-01                 1514.68
2 2013-03-01                 1569.19
          Date  Adjusted_Closing_Price
137 2024-06-01                 5460.48
138 2024-07-01                 5522.30
139 2024-08-01                 5648.40


## U.S. Exports of Goods by F.A.S. Basis to Russia_FRED

**Exports of Goods**: Refers to all goods (physical products) that are shipped from the United States to Russia. This includes items such as machinery, agricultural products, technology, and more.


F.A.S. (Free Alongside Ship) Basis: The F.A.S. value represents the cost of the exported goods up to the point where they are delivered alongside the ship at the U.S. port of export. It does not include costs associated with shipping, insurance, or any charges incurred after the goods have been placed alongside the vessel.


**Why Is This Indicator Important?**
- Economic Relationships:This indicator provides insights into the trade relationship between the United States and Russia. Monitoring export trends can help understand economic interdependencies and the impact of geopolitical events on trade flows.
- Impact of Geopolitical Events:Events like the Russia-Ukraine conflict can lead to sanctions, trade restrictions, or changes in export volumes. Tracking U.S. exports to Russia helps assess the economic impact of such events on trade.


**Use in Economic Analysis:**
This data is useful for analyzing how international trade dynamics influence the U.S. economy, especially in response to policy changes or geopolitical tensions. It can also be relevant for studying the impact of trade disruptions on U.S. industries that heavily export to Russia.
Use in Your Econometric Model. If your project aims to study the economic impact of the Russia-Ukraine conflict on the U.S. economy, the U.S. Exports to Russia variable can be crucial. It provides a direct measure of how trade flows have been affected, which may have ripple effects on economic indicators like GDP, employment, or industry performance.

In [38]:
df9 = pd.read_csv('U.S. Exports of Goods by F.A.S. Basis to Russia_FRED.csv')
df9.head(3)

,DATE,EXP4621
0,1992-01-01,1.3
1,1992-02-01,66.4
2,1992-03-01,161.3


In [39]:
df_US_Export_Goods_Russia = preprocess_and_filter_dates(df9,
                                               date_column='DATE',
                                               EXP4621='US_Export_to_Russia')
                                               # drop_columns=['Open', 'High', 'Low', 'Close', 'Volume'])

In [40]:
print(df_US_Export_Goods_Russia.head(3))
print(df_US_Export_Goods_Russia.tail(3))

        Date  US_Export_to_Russia
0 2013-01-01           692.919960
1 2013-02-01          1008.624490
2 2013-03-01           917.892441
          Date  US_Export_to_Russia
137 2024-06-01            42.336284
138 2024-07-01            45.282786
139 2024-08-01            54.541501


## Unemployment Rate

**1. Economic Health Indicator**
The unemployment rate reflects the overall health of the economy. High unemployment indicates economic distress, while low unemployment suggests economic strength. By including the unemployment rate in your model, you can analyze how external shocks, such as geopolitical conflicts, influence the labor market.

**2. Link to Economic Activity**
Changes in the unemployment rate can impact consumer spending, business investment, and economic growth. For instance, if the Russia-Ukraine conflict leads to economic disruptions, you may observe a rise in unemployment, which could negatively affect GDP and economic stability.
The unemployment rate can provide insights into how labor market dynamics interact with other economic indicators like inflation, consumer sentiment, and industrial output.

**3. Interaction with Inflation (Phillips Curve Relationship)**
There is a well-known relationship between unemployment and inflation, often described by the Phillips Curve. Typically, lower unemployment leads to higher inflation as increased demand for goods and services drives prices up. Conversely, higher unemployment often leads to lower inflation.
By including the unemployment rate in your model, you can better understand how labor market conditions influence inflation dynamics, especially in the context of external shocks like the conflict.

**4. Impact of Geopolitical Events**
Geopolitical events like the Russia-Ukraine conflict can have both direct and indirect effects on the U.S. labor market. For example:
Direct Impact: Industries that rely on trade with Russia may experience job losses.
Indirect Impact: Higher energy prices or supply chain disruptions could affect business operations and employment levels.
By including the unemployment rate in your VAR model, you can examine how these impacts unfold over time and how they interact with other economic variables.

**5. Policy Implications**
The unemployment rate is closely monitored by policymakers, including the Federal Reserve, when making decisions about monetary policy. If the conflict leads to higher unemployment, it could prompt the Fed to adjust interest rates or implement other measures to stabilize the economy.
Analyzing the unemployment rate alongside other variables can provide insights into the effectiveness of policy responses.


**Use in Your Econometric Model**
Including the unemployment rate in your VAR model allows you to capture how labor market conditions respond to external shocks and how these changes influence other economic indicators like inflation, GDP, and consumer sentiment.
It also helps in understanding the short-term and long-term effects of the conflict on the U.S. economy, providing a more comprehensive analysis of the economic impact.

In [41]:
df10 = pd.read_csv('Unemployment Rate_FRED.csv')
df10.head(3)

,DATE,UNRATE
0,1948-01-01,3.4
1,1948-02-01,3.8
2,1948-03-01,4.0


In [42]:
df_US_Unemployment_Rate = preprocess_and_filter_dates(df10,
                                               date_column='DATE',
                                               UNRATE='US_Unemployment_Rate')

In [43]:
print(df_US_Unemployment_Rate.head(3))
print(df_US_Unemployment_Rate.tail(3))

        Date  US_Unemployment_Rate
0 2013-01-01                   8.0
1 2013-02-01                   7.7
2 2013-03-01                   7.5
          Date  US_Unemployment_Rate
137 2024-06-01                   4.1
138 2024-07-01                   4.3
139 2024-08-01                   4.2


## US Consumer Sentiment 

The University of Michigan Consumer Sentiment Index measures consumer confidence in the economy based on surveys of households about their financial conditions and future expectations. It's a key indicator of consumer spending, which drives a significant portion of economic activity in the U.S.

**Importance for Econometric Model**:
- Gauge of Consumer Behavior: It helps assess how geopolitical events like the Russia-Ukraine conflict impact consumer confidence, which can influence spending and overall economic growth.
- Early Economic Indicator: Changes in consumer sentiment often precede shifts in economic conditions, making it useful for understanding potential future trends in GDP, inflation, and employment.
- Interaction with Other Variables: In a VAR model, it allows for analyzing how consumer sentiment interacts with other economic indicators, providing insights into the ripple effects of economic shocks.
- Incorporating this index can enhance your model's ability to capture the impact of external events on consumer-driven economic dynamics.

In [44]:
df11 = pd.read_csv('University of Michigan Consumer Sentiment_FRED.csv')
df11.head(3)

,DATE,UMCSENT
0,2012-01-01,75.0
1,2012-02-01,75.3
2,2012-03-01,76.2


In [45]:
df_US_Consumer_Sentiment = preprocess_and_filter_dates(df11,
                                               date_column='DATE',
                                               UMCSENT='US_Consumer_Sentiment')

In [46]:
print(df_US_Consumer_Sentiment.head(3))
print(df_US_Consumer_Sentiment.tail(3))

        Date  US_Consumer_Sentiment
0 2013-01-01                   73.8
1 2013-02-01                   77.6
2 2013-03-01                   78.6
          Date  US_Consumer_Sentiment
137 2024-06-01                   68.2
138 2024-07-01                   66.4
139 2024-08-01                   67.9


## US Gasoline Prices

Monthly gasoline prices per barrel are relevant to your econometric model because they reflect the cost of energy, which is a major component of consumer and producer expenses. Here’s why they are important:

- Impact on Inflation: Gasoline prices directly affect transportation and production costs, which can lead to higher consumer prices (inflation). Tracking these prices helps understand how energy costs contribute to overall inflation, especially during events like the Russia-Ukraine conflict, which disrupts global energy markets.

- Consumer Spending: Higher gasoline prices reduce disposable income, leading to lower consumer spending, which impacts economic growth. By including gasoline prices, your model can capture this economic relationship.

- Indicator of Economic Shocks: Sudden increases in gasoline prices often signal economic shocks or supply disruptions. They can reveal how external events (e.g., geopolitical conflicts) impact the economy.

In [47]:
df12 = pd.read_excel('US Gasoline Prices Monthly_US energy information administration(EIA).xls', sheet_name='Data 1')
df12.head(3)

,Date,U.S. Regular All Formulations Retail Gasoline Prices (Dollars per Gallon)
0,1990-08-15,NaN
1,1990-09-15,1.258
2,1990-10-15,1.335


In [48]:
df12.columns

Index(['Date', 'U.S. Regular All Formulations Retail Gasoline Prices (Dollars per Gallon)'], dtype='object')

In [49]:
df12.rename(columns={'U.S. Regular All Formulations Retail Gasoline Prices (Dollars per Gallon)':'US_Gasoline_Dollars_Per_Barrel'},
            inplace=True)

In [50]:
df_Gasoline_Dollar_Per_Barrel = preprocess_and_filter_dates(df12,
                                                           date_column = 'Date',)

In [51]:
print(df_Gasoline_Dollar_Per_Barrel.head(3))

        Date  US_Gasoline_Dollars_Per_Barrel
0 2013-01-01                           3.319
1 2013-02-01                           3.670
2 2013-03-01                           3.711


## USD to Rubble Data

Monthly gasoline prices per barrel are relevant to your econometric model because they reflect the cost of energy, which is a major component of consumer and producer expenses. Here’s why they are important:

- Impact on Inflation: Gasoline prices directly affect transportation and production costs, which can lead to higher consumer prices (inflation). Tracking these prices helps understand how energy costs contribute to overall inflation, especially during events like the Russia-Ukraine conflict, which disrupts global energy markets.

 - Consumer Spending: Higher gasoline prices reduce disposable income, leading to lower consumer spending, which impacts economic growth. By including gasoline prices, your model can capture this economic relationship.

- Indicator of Economic Shocks: Sudden increases in gasoline prices often signal economic shocks or supply disruptions. They can reveal how external events (e.g., geopolitical conflicts) impact the economy.




**Why "Price"?**
- Represents Energy Costs: The "Price" column reflects the cost per barrel of gasoline, which is a critical component of energy expenses in the economy.
- Impact on Inflation: Fluctuations in gasoline prices have a direct effect on inflation, as energy costs influence the prices of goods and services across the economy.
- Influence on Consumer Spending: Changes in gasoline prices affect consumers' disposable income, impacting overall consumer spending and economic activity.
- Energy Market Dynamics: Including gasoline prices in your model will help capture the impact of energy market disruptions, such as those caused by the Russia-Ukraine conflict.

In [52]:
df13 = pd.read_csv('USD_RUB Historical Data_Investing.com.csv')
df13.head(3)

,Date,Price,Open,High,Low,Vol.,Change %
0,11/1/2024,98.1972,97.3723,100.4089,95.4790,NaN,0.85%
1,10/1/2024,97.3705,92.9100,100.3000,92.7000,NaN,4.70%
2,9/1/2024,93.0009,91.5385,94.3637,87.0744,NaN,2.59%


In [53]:
df13.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'], dtype='object')

In [54]:
df_USD_Rubble_Conv = preprocess_and_filter_dates(df13,
                                               date_column='Date',
                                               EXP4621='US_Export_to_Russia',
                                               drop_columns=['Open', 'High', 'Low', 'Vol.', 'Change %'])

In [55]:
print(df_USD_Rubble_Conv.head(3))
print(df_USD_Rubble_Conv.tail(3))

        Date   Price
0 2013-01-01  30.030
1 2013-02-01  30.650
2 2013-03-01  31.078
          Date    Price
137 2024-06-01  85.5332
138 2024-07-01  85.9988
139 2024-08-01  90.6509


## Volatility Index

For the Volatility Index (VIX) data, the "CLOSE" column is typically the variable of interest for our model.

**Why "CLOSE"?**
- The CLOSE value represents the final VIX value for each trading day, giving a summary measure of market volatility for that day. In your econometric model, you are likely working with monthly data, so you would typically aggregate the daily VIX data to a monthly level (e.g., using the average or end-of-month closing value).

**Importance of VIX Data for Your Model:**
- Indicator of Market Sentiment: The VIX measures the market's expectations of near-term volatility, often referred to as the "fear gauge." High VIX values indicate high market uncertainty, while low values suggest market stability.
- Impact of Geopolitical Events: The Russia-Ukraine conflict may have increased market uncertainty, affecting investor behavior and economic conditions. The VIX can help quantify the market's reaction to such geopolitical shocks.
- Interaction with Economic Variables: Including the VIX in your VAR model can help capture the relationship between market volatility and other economic indicators like inflation, gasoline prices, and consumer sentiment, providing insights into how financial markets respond to economic disruptions.

In [56]:
df14 =pd.read_csv('VIX_History_CBOE Global Markets.csv')
df14.head(3)

,DATE,OPEN,HIGH,LOW,CLOSE
0,01/02/1990,17.24,17.24,17.24,17.24
1,01/03/1990,18.19,18.19,18.19,18.19
2,01/04/1990,19.22,19.22,19.22,19.22


In [57]:
df14['DATE'] = pd.to_datetime(df14['DATE']) 

# Extract the year and month from the DATE
df14['YearMonth'] = df14['DATE'].dt.to_period('M')

# Group by YearMonth and aggregate
monthly_df = df14.groupby('YearMonth').agg({
    'CLOSE': 'mean'  # Use 'mean' for the average VIX value per month
}).reset_index()

# Convert YearMonth back to a datetime object (first day of the month)
monthly_df['DATE'] = monthly_df['YearMonth'].dt.to_timestamp()

# Drop the YearMonth column 
monthly_df = monthly_df.drop(columns=['YearMonth'])
print(monthly_df.head())

       CLOSE       DATE
0  23.347273 1990-01-01
1  23.262632 1990-02-01
2  20.062273 1990-03-01
3  21.403500 1990-04-01
4  18.097727 1990-05-01


In [58]:
df_US_Market_Monthly_Volatility = preprocess_and_filter_dates(monthly_df,
                                               date_column='DATE',
                                               CLOSE='US_Mean_Monthly_Market_Volatility')

In [59]:
print(df_US_Market_Monthly_Volatility.head(3))
print(df_US_Market_Monthly_Volatility.tail(3))

   US_Mean_Monthly_Market_Volatility       Date
0                          13.505238 2013-01-01
1                          14.072632 2013-02-01
2                          13.031500 2013-03-01
     US_Mean_Monthly_Market_Volatility       Date
137                          12.667000 2024-06-01
138                          14.373913 2024-07-01
139                          19.312273 2024-08-01


# Consolidated Dataset for Modelling

In [62]:
dataframes = [df_Global_Shipping_Cost,
                df_Crude_Oil_Prices_Monthly,
                df_Federal_Funds_Rate,
                df_inflation_cpi,
                df_nominal_broad_usd_index,
                df_producer_price_index_all_commodities,
                df_sp500,
                df_US_Export_Goods_Russia,
                df_US_Unemployment_Rate,
                df_US_Consumer_Sentiment,
                df_Gasoline_Dollar_Per_Barrel,
                df_USD_Rubble_Conv,
                df_US_Market_Monthly_Volatility]

In [63]:
# Start with the first DataFrame
merged_df = df_Avg_Hourly_Earning

# Use a loop to join all other DataFrames on the Date column
for df in dataframes:
    merged_df = merged_df.join(df.set_index('Date'), on='Date', how='outer')

merged_df.head(5)

,Date,Average_Hourly_Earning,Trade_Volume_Pct_Change_Index_Value,US_Crude_Oil_Prices_Dollar_Per_Barrel,Federal_Rates_Monthly,CPI_Value_Inflation,Nominal_Broad_US_Dollar_Index,producer_price_index_all_commodities,Adjusted_Closing_Price,US_Export_to_Russia,US_Unemployment_Rate,US_Consumer_Sentiment,US_Gasoline_Dollars_Per_Barrel,Price,US_Mean_Monthly_Market_Volatility
0,2013-01-01,23.75,0.004045,95.00,0.14,230.280,90.8255,202.5,1498.11,692.919960,8.0,73.8,3.319,30.030,13.505238
1,2013-02-01,23.78,-0.000250,95.01,0.15,232.166,91.5509,204.3,1514.68,1008.624490,7.7,77.6,3.670,30.650,14.072632
2,2013-03-01,23.80,0.009400,95.54,0.14,232.773,92.4132,204.0,1569.19,917.892441,7.5,78.6,3.711,31.078,13.031500
3,2013-04-01,23.87,-0.002476,94.41,0.15,232.531,92.0478,203.5,1597.57,1087.226442,7.6,76.4,3.570,31.128,13.967273
4,2013-05-01,23.89,-0.003095,94.75,0.11,232.945,92.5290,204.1,1630.74,781.571515,7.5,84.5,3.615,31.922,13.493636


In [64]:
# Create the Russia Ukraine War Flag
merged_df['Russia_Ukraine_Date_Flag'] = (merged_df['Date'] >= pd.Timestamp('2022-02-01')).astype(int) 

In [67]:
# Create the Covid Flag
# Start Date - January 20, 2020
# End Date - May 11, 2023

# Create the Covid Flag
merged_df['Covid_Flag'] = (
    (merged_df['Date'] >= pd.Timestamp('2020-01-20')) & 
    (merged_df['Date'] <= pd.Timestamp('2023-05-11'))
).astype(int)

In [68]:
merged_df.head()

,Date,Average_Hourly_Earning,Trade_Volume_Pct_Change_Index_Value,US_Crude_Oil_Prices_Dollar_Per_Barrel,Federal_Rates_Monthly,CPI_Value_Inflation,Nominal_Broad_US_Dollar_Index,producer_price_index_all_commodities,Adjusted_Closing_Price,US_Export_to_Russia,US_Unemployment_Rate,US_Consumer_Sentiment,US_Gasoline_Dollars_Per_Barrel,Price,US_Mean_Monthly_Market_Volatility,Russia_Ukraine_Date_Flag,Covid_Flag
0,2013-01-01,23.75,0.004045,95.00,0.14,230.280,90.8255,202.5,1498.11,692.919960,8.0,73.8,3.319,30.030,13.505238,0,0
1,2013-02-01,23.78,-0.000250,95.01,0.15,232.166,91.5509,204.3,1514.68,1008.624490,7.7,77.6,3.670,30.650,14.072632,0,0
2,2013-03-01,23.80,0.009400,95.54,0.14,232.773,92.4132,204.0,1569.19,917.892441,7.5,78.6,3.711,31.078,13.031500,0,0
3,2013-04-01,23.87,-0.002476,94.41,0.15,232.531,92.0478,203.5,1597.57,1087.226442,7.6,76.4,3.570,31.128,13.967273,0,0
4,2013-05-01,23.89,-0.003095,94.75,0.11,232.945,92.5290,204.1,1630.74,781.571515,7.5,84.5,3.615,31.922,13.493636,0,0


# Export the Dataset

In [69]:
merged_df.to_csv('Econ_Project_Data_Consolidated.csv', index=False)